In [68]:
import numpy as np
import pandas as pd
import vectorbt as bt
import yfinance as yf
import pandas_ta as ta

In [69]:
btc = yf.download('BTC-USD', start='2024-01-01', end='2024-12-05', interval='1h')
btc.index = pd.to_datetime(btc.index)  # Ensure the index is datetime
btc.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
btc = btc.asfreq('h')
btc

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Datetime,,,,,,
2024-01-01 00:00:00+00:00,42477.253906,42477.253906,42517.601562,42277.726562,42280.234375,0
2024-01-01 01:00:00+00:00,42622.898438,42622.898438,42718.710938,42433.785156,42471.402344,200035328
2024-01-01 02:00:00+00:00,42576.601562,42576.601562,42611.335938,42506.359375,42611.335938,0
2024-01-01 03:00:00+00:00,42320.730469,42320.730469,42581.007812,42270.757812,42575.968750,118417408
2024-01-01 04:00:00+00:00,42387.164062,42387.164062,42387.164062,42227.984375,42325.042969,34720768
...,...,...,...,...,...,...
2024-12-04 19:00:00+00:00,97306.421875,97306.421875,97306.421875,96210.468750,96269.585938,1414168576
2024-12-04 20:00:00+00:00,98956.718750,98956.718750,99207.328125,97312.203125,97312.203125,7638441984
2024-12-04 21:00:00+00:00,97912.992188,97912.992188,99095.601562,97685.226562,98948.906250,2957557760


In [70]:
# Initial parameters
initial_capital = 100000
risk_per_trade = 0.02  # 2%
stop_loss = 0.18  # 18%
take_profit = 0.40  # 40%

In [71]:
type(btc['Close'])

pandas.core.series.Series

In [72]:
# btc['rsi']= ta.rsi(btc['Close'], length=14)
# btc['ema_short'] = ta.ema(btc['Close'], length=20)
# btc['ema_long'] = ta.ema(btc['Close'], length=50)

# btc['long_entry'] = ((btc['ema_short'] > btc['ema_long']) & (btc['rsi'] < 35)).astype(int)
# btc['long_exit'] = ((btc['rsi']>70)).astype(int)

# btc['short_entry'] = ((btc['ema_long'] > btc['ema_short']) & (btc['rsi'] > 65)).astype(int)
# btc['short_exit'] = (btc['rsi']<30).astype(int)

# btc.tail()

In [79]:
signal = pd.read_csv('yfp_series.csv')
signal.index = pd.to_datetime(signal['Datetime'])

btc['signal'] = signal['Predictions']

btc['rsi']= ta.rsi(btc['Close'], length=14)
btc['ema_short'] = ta.ema(btc['Close'], length=20)
btc['ema_long'] = ta.ema(btc['Close'], length=50)

print(btc)
# Create long and short signals
# long_signal = ((signal['Predictions'] == 1) & (btc['rsi'] < 30) & (btc['ema_short'] > btc['ema_long'])).astype(int)
# short_signal = ((signal['Predictions'] == 0) & (btc['rsi'] > 70) & (btc['ema_short'] < btc['ema_long']) ).astype(int)

long_signal = ((signal['Predictions'] == 1) & (btc['rsi'] < 45)).astype(int)
short_signal = ((signal['Predictions'] == 0) & (btc['rsi'] > 55)).astype(int)


long_signal.index = pd.to_datetime(long_signal.index)
short_signal.index = pd.to_datetime(short_signal.index)

# print(btc['Close'].index)
# print(long_signal.index)
# print(short_signal.index)




                              Adj Close         Close          High  \
Datetime                                                              
2024-01-01 00:00:00+00:00  42477.253906  42477.253906  42517.601562   
2024-01-01 01:00:00+00:00  42622.898438  42622.898438  42718.710938   
2024-01-01 02:00:00+00:00  42576.601562  42576.601562  42611.335938   
2024-01-01 03:00:00+00:00  42320.730469  42320.730469  42581.007812   
2024-01-01 04:00:00+00:00  42387.164062  42387.164062  42387.164062   
...                                 ...           ...           ...   
2024-12-04 19:00:00+00:00  97306.421875  97306.421875  97306.421875   
2024-12-04 20:00:00+00:00  98956.718750  98956.718750  99207.328125   
2024-12-04 21:00:00+00:00  97912.992188  97912.992188  99095.601562   
2024-12-04 22:00:00+00:00  98403.640625  98403.640625  98697.328125   
2024-12-04 23:00:00+00:00  98799.390625  98799.390625  99122.093750   

                                    Low          Open      Volume  signal  \

In [81]:
pf = bt.Portfolio.from_signals(btc['Close'], entries=long_signal, short_entries=short_signal,fees=0.000, slippage=0.000, sl_stop=0.05, tp_stop=0.025)

In [82]:
returns = pf.total_return()
print(returns)

0.09224434099841673


In [83]:
pf.stats()

Start                         2024-01-01 00:00:00+00:00
End                           2024-12-04 23:00:00+00:00
Period                                339 days 00:00:00
Start Value                                       100.0
End Value                                    109.224434
Total Return [%]                               9.224434
Benchmark Return [%]                         132.593639
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              44.560101
Max Drawdown Duration                 103 days 20:00:00
Total Trades                                        231
Total Closed Trades                                 230
Total Open Trades                                     1
Open Trade PnL                                -3.042242
Win Rate [%]                                  67.826087
Best Trade [%]                                 4.651419
Worst Trade [%]                               -7

In [84]:
pf.plot(
    subplots=['trades', 'orders', 'trade_pnl', 'cum_returns']
    )


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '01a040a6-20ad-4e20-9f83-c203a848f9c1',
              'x': array([datetime.datetime(2024, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 1, 1, 1, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 1, 1, 2, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2024, 12, 4, 21, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 22, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 4, 23, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([42477.25390625, 42622.8984375 , 42576.6015625 , ..., 97912.9921875 ,
               

In [78]:
# pf.plot().show()